In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
seg_runs = [
    ('8bh5rurv', 0.8848695158958435),
    ('cnqyhmsb', 0.8836771249771118),
    ('ri1dvvam', 0.8832937479019165),
    ('ilhz9vkq', 0.883250892162323),
    ('dhmhz041', 0.8829216957092285),
    ('7mrir9a3', 0.8828873634338379),
    ('yzmxfvrv', 0.8828799724578857),
    ('5ywrt6qn', 0.882686972618103),
    ('ighbwtdm', 0.8826081156730652),
    ('315hd0sm', 0.8825200796127319),
    ('y2jgqdox', 0.8821526169776917),
    ('xr44njvv', 0.8815780282020569),
    ('mb22v5ps', 0.8812705874443054),
    ('q6d1n08d', 0.8809899091720581),
    ('37y0hk18', 0.8808367848396301),
    ('3ag6fjep', 0.8806321024894714),
    ('uevn0e5g', 0.8803757429122925),
    ('0en8eomx', 0.8799382448196411),
    ('o0wao11j', 0.8796645998954773),
    ('4qrallkn', 0.8785540461540222),
    ('6vya7uxd', 0.8780744671821594),
    ('rwa0acur', 0.8775778412818909),
    ('qoijsx0h', 0.8775392770767212),
    ('l1hf5uuh', 0.8767494559288025),
    ('ebwyd2di', 0.8765263557434082),
    ('6g0nkv0j', 0.8760395646095276),
    ('ph4c64ar', 0.8760202527046204),
    ('amkk3cbw', 0.8735491037368774),
    ('c1dlbdub', 0.8734628558158875),
    ('hiinbqsu', 0.8719046115875244)
]

seg_dam_ids = [
    ('tyi7q17m', 0.8150793196256518),
    ('ngmxec6s', 0.8096530334019485),
    ('vrgbzaup', 0.8096383585083926),
    ('443t9312', 0.8083381310832123),
    ('pxcuoulb', 0.8081565006839244),
    ('cpumt46r', 0.8080391959243641),
    ('viptnxip', 0.8076960005628842),
    ('mhczmsoa', 0.8073159432491346),
    ('znxzcx0a', 0.8072744354969165),
    ('7ar6am07', 0.8071092834046755),
    ('lwe6n5c0', 0.8069356023059625),
    ('90mnu920', 0.8069068799555296),
    ('08meqrfc', 0.806507592996532),
    ('j4xmz3we', 0.8061300715307924),
    ('5ocdy37k', 0.806018066403344),
    ('5pn3bp2o', 0.8057287190805172),
    ('0gvvydkt', 0.8057094304994589),
    ('or34wjnl', 0.8056978455958853),
    ('8n17nhye', 0.8053004703929182),
    ('c2hvohg7', 0.8050853487741387),
    ('33aasxzr', 0.8048996430920557),
    ('40l1n8y6', 0.804133361539285),
    ('8slg69rb', 0.8039897395816178),
    ('yzy5xz0v', 0.8037504662087867),
    ('71urz3vc', 0.803504510967261),
    ('plef5k0s', 0.8033634254385499),
    ('rfu2ifuh', 0.8031063433726832),
    ('5zu1m3bn', 0.8028573049710269),
    ('nj4eqar6', 0.8022980617768509),
    ('95pmh37c', 0.8022721609563958)
]

building_damage = [
    ('kx9yzcb9', 0.8188149291818385),
    ('wbqskbmu', 0.8104202275388073),
    ('y7n4yzya', 0.808878301295482),
    ('c1lit0gf', 0.8086308781844662),
    ('ygnsmpis', 0.8047019871807307),
    ('t91jg9hv', 0.8030884276616155),
    ('5v3elkfr', 0.8026197042480495),
    ('0qjy2h9e', 0.8011403695871695),
    ('6s629jji', 0.7983212991363049),
    ('umj3wmwh', 0.7972560052280561),
    ('phtv4ekc', 0.7952364835570059),
    ('ypydqpw5', 0.7950183345724261),
    ('9bv7h3cw', 0.7941133157892892),
    ('w8ka9fat', 0.7932566433728835),
    ('zu0uhkyk', 0.7921109978206695)
]

In [ ]:
import wandb
api = wandb.Api()
def get_score(r, metric):
    hist = list(r.scan_history())
    scores = [h[METRIC] for h in hist if METRIC in h]
    if not scores: return None
    return max(scores)

In [ ]:
import wandb
from xv import io
from xv.io import Config
import torch
from tqdm.notebook import tqdm
from datetime import datetime
import os

In [ ]:
dmg_is_seg = True

seg_run_id = "qoijsx0h"
dmg_run_id = "0gvvydkt"

In [ ]:
from datetime import datetime
import os
test_dir = '../../datasets/xview/test'
submission_dir = f"../submissions/{seg_run_id}_{dmg_run_id}_{str(datetime.now()).replace(' ', '_').replace(':', '-')}"

cache_dir = f'../submissions/cache/{seg_run_id}'
os.mkdir(submission_dir)
print(submission_dir)

In [ ]:
import shutil

In [ ]:
cache_dir

In [ ]:
import ttach as tta
from glob import glob
from PIL import Image
import numpy as np

USE_CACHED_SEG = True
CACHE_SEG = False

if USE_CACHED_SEG:
    assert os.path.isdir(cache_dir)
    !cp {cache_dir}/* {submission_dir}
    
else:
    if CACHE_SEG:
        if os.path.isdir(cache_dir):
            shutil.rmtree(cache_dir)
        os.mkdir(cache_dir)

    seg_run_path = f"xvr-hlt/sky-eye-full/{seg_run_id}"
    conf_file = wandb.restore('config.yaml', run_path=seg_run_path, replace=True).name
    state_file = wandb.restore('state_dict.pth', run_path=seg_run_path, replace=True).name

    conf = Config(conf_file)
    model, preprocess_fn = io.load_segmentation_model(conf, state_file)
    model = model.eval().cuda()
    model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')
    pre_files = glob(f"{test_dir}/images/*pre*")
    
    with torch.no_grad():
        for f in tqdm(pre_files):
            i = io.load_img(f, preprocess_fn)
            out = model(i.cuda())[0][0] > 0.
            out = out.cpu().numpy().astype(np.uint8)
            fid = f.split('_')[-1].replace(".png", "")
            Image.fromarray(out).save(f"{submission_dir}/test_localization_{fid}_prediction.png")
            if CACHE_SEG:
                Image.fromarray(out).save(f"{cache_dir}/test_localization_{fid}_prediction.png")

In [ ]:
from xv.tta import BoxClassifierTTA
import ttach as tta

if dmg_is_seg:
    dmg_run_path = f"xvr-hlt/building-seg-damage/{dmg_run_id}"
else:
    dmg_run_path = f"xvr-hlt/building-damage/{dmg_run_id}"

dmg_conf_file = wandb.restore('config.yaml', run_path=dmg_run_path, replace=True).name
dmg_state_file = wandb.restore('state_dict.pth', run_path=dmg_run_path, replace=True).name

dmg_conf = Config(dmg_conf_file)

if dmg_is_seg:
    dmg_model, dmg_preprocess_fn = io.load_segmentation_model(dmg_conf, dmg_state_file)
    dmg_model = tta.SegmentationTTAWrapper(dmg_model, tta.aliases.d4_transform(), merge_mode='mean')
else:
    dmg_model = io.load_damage_model(dmg_conf, dmg_state_file)
    dmg_model = BoxClassifierTTA(dmg_model)
    
dmg_model = dmg_model.eval().cuda()

In [ ]:
from glob import glob 
from PIL import Image
import numpy as np
import cv2
from imantics import Polygons

post_files = glob(f"{test_dir}/images/*post*")

In [ ]:
@torch.no_grad()
def get_box_damage(dmg_model, inp_file, mask_file):
    msk_dmg = np.zeros((1024, 1024), dtype=np.uint8)
    
    msk = np.array(Image.open(mask_file))
    polypoints = polys.points
    polys = Polygons.from_mask(msk)
    
    if not polypoints:
        return Image.fromarray(msk_dmg)
    
    inp = io.load_dmg_img(inp_file)
    boxes = torch.Tensor([[min(p[:,0]), min(p[:,1]), max(p[:,0]), max(p[:,1])] for p in polypoints])
    out = dmg_model(inp.cuda(), [boxes.cuda()])
    classes = (out.argmax(1) + 1).cpu().numpy()

    for poly, cls in zip(polypoints, classes):
        cv2.fillPoly(msk_dmg, [poly], int(cls))

    return Image.fromarray(msk_dmg)

@torch.no_grad()
def get_seg_damage(dmg_model, preprocess_fn, inp_file, mask_file):
    inp = io.load_img(f, dmg_preprocess_fn)
    out = dmg_model(inp.cuda())[0]
    out = (out.argmax(0) + 1).cpu().numpy()
    return Image.fromarray(out.astype(np.uint8))

@torch.no_grad()
def get_seg_damage(dmg_model, preprocess_fn, inp_file, mask_file):
    mb = np.array(Image.open(mask_file))
    inp = io.load_img(f, dmg_preprocess_fn)
    out = dmg_model(inp.cuda())[0]
    
    out = out.sigmoid()/out.sigmoid().sum(0)
    out = out.cpu().numpy()
    
    damage_map = np.zeros((1024,1024))
    
    for poly in Polygons.from_mask(mb):
        poly_mask = Polygons.create([poly]).mask(1024, 1024).array
        cls = out[:,poly_mask].mean(1).argmax() + 1
        damage_map[poly_mask] = cls
    
    return Image.fromarray(damage_map.astype(np.uint8))

In [ ]:
for f in tqdm(post_files):
    fid = f.split('_')[-1].replace(".png", "")
    mask_file = f"{submission_dir}/test_localization_{fid}_prediction.png"
    
    if dmg_is_seg:
        img = get_seg_damage(dmg_model, dmg_preprocess_fn, f, mask_file)
    else:
        img = get_box_damage(dmg_model, f, mask_file)

    img.save(f"{submission_dir}/test_damage_{fid}_prediction.png")

In [ ]:
_split = submission_dir.rfind('/') + 1
root, subdir = submission_dir[:_split], submission_dir[_split:]

! cd "{root}" && zip -r "{subdir}.zip" "{subdir}/"

In [ ]:
!rm config.yaml state_dict.pth